In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'
ab_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base'

In [2]:
#stationär
data_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\data'
ab_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\base'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [11]:
# Budget config file
include_pheno = False
threshold_year = 1970
data_path = data_dir
ab_path = ab_dir
max_length = [51,81]
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
dim_emb = 128
dim_hidden = 128
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 5


Vocabulary

In [ ]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
include_pheno = False
vocabulary = vocab_geno(NCBI, include_pheno)
vocab = vocab_pheno(ab_df)
print(len(vocabulary))
print(len(vocab))


In [16]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

data_path = data_dir
ab_path = ab_dir

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,81]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [ ]:
sequences = deepcopy(NCBI['AST_phenotypes'].tolist())
max_seq_len = [max_length[0],max_length[1]]
list_idx = []
list_SR = []
for i in range(len(sequences)):
    current_seq = sequences[i]
    current_idxs = []
    current_SRs = []
    for j in range(len(current_seq)):
        item = current_seq[j].split('=')
        abs = item[0]   
        sr = item[1]
        current_idxs.append(vocabulary_pheno.lookup_indices([abs]))
        for k in range(len(sr)):
            if sr == 'R':
                current_SRs.append(1)
            else:
                current_SRs.append(0)

    if len(current_idxs) != len(current_SRs):
        print("current sequence:",current_seq, "\n", "with length:", len(current_seq))
        print("indexes:",current_idxs, "with length:", len(current_idxs))
        print("suceptability",current_SRs, "with length:", len(current_SRs))
        print('error at', j)
        print("--------------------")
    current_idxs = [int(item[0]) for item in current_idxs]
    #for i in range(0,max_length[1] - len(current_idxs)):
    #    current_idxs.append(-1)
    #for i in range(0,max_length[1] - len(current_SRs)):
    #    current_SRs.append(-1)
    list_idx.append(current_idxs)
    list_SR.append(current_SRs)
for i in range(len(list_idx)):
    if len(list_idx[i]) != len(list_SR[i]):
        print('error at', i)

In [22]:
import torch
import torch.nn.functional as F

def custom_loss(logits, targets, pad_index=-1):
    #print(logits)
    #print(targets)
    loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
    #print(loss)
    mask = (targets != pad_index).float()
    #print(mask)
    masked_loss = loss * mask
    #print(masked_loss)
    average_loss = masked_loss.sum() / mask.sum()
    #print(average_loss)
    return average_loss


In [37]:
from torch.utils.data import DataLoader 
from bert_builder import BERT_ft
import torch.nn as nn
import copy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_function = torch.nn.BCEWithLogitsLoss()
bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

optimizer = torch.optim.AdamW(bert_test.parameters(), lr=0.001, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)


loader = DataLoader(test_set, batch_size=16, shuffle=False)
for i, batch in enumerate(loader):
    if i >= 10:
        break 
    optimizer.zero_grad()
    input, token_target, attn_mask, AB_idx, SR_class = batch
    token_predictions, resistance_predictions = bert_test(input, attn_mask) 
    result_list = []
    for j in range(len(AB_idx)):
        result_tensor = torch.full((81,), -1)  # Create tensor filled with -1 values
        for idx, value in enumerate(AB_idx[j]):
            if value != -1:
                result_tensor[value.item()] = SR_class[j][idx]
        result_list.append(result_tensor)
    ab_loss = 0
    pheno_loss = 0
    for i, row in enumerate(resistance_predictions):
        prediction = row
        target = result_list[i]
        ab_loss = custom_loss(prediction, target.float()) 
        pheno_loss += ab_loss
    params_before_optimization = copy.deepcopy(bert_test.state_dict())
    pheno_loss.backward()
    optimizer.step()
    
    
    total_correct = 0
    total_sum = 0
    list_AB_predictions = []
    pred_res = torch.where(resistance_predictions > 0, torch.ones_like(resistance_predictions), torch.zeros_like(resistance_predictions))
    for i, row in enumerate(pred_res):
        AB_list = 0
        AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
        current_abs = []
        for ab in AB_list:
            current_abs.append(row[ab].item())
        current_abs = torch.tensor(current_abs)
        current_abs = current_abs.type(torch.int16)
        list_AB_predictions.append(current_abs)
    
        processed_tensor = [row[row != -1] for row in SR_class]
    for i, row in enumerate(processed_tensor):
        print(row)
        print(list_AB_predictions[i])
        total_correct += (row == list_AB_predictions[i]).sum().item()
        print(total_correct)
        total_sum += len(row)
    acc = total_correct / total_sum 
    print(acc)

    


tensor([1])
tensor([0], dtype=torch.int16)
0
tensor([1])
tensor([0], dtype=torch.int16)
0
tensor([0])
tensor([0], dtype=torch.int16)
1
tensor([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0])
tensor([1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       dtype=torch.int16)
13
tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0])
tensor([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
        1], dtype=torch.int16)
27
tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0])
tensor([1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1], dtype=torch.int16)
35
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0])
tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=torch.int16)
41
tensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0])
tensor([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], dtype=torch.int16)
49
tensor([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0])
tensor([1, 1, 0, 0, 1, 1

Trainer

In [40]:
import time
from torch import nn
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
import copy
import torch.nn.functional as F

class BertTrainer_ft:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.1
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        self.device = device
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.token_criterion = nn.CrossEntropyLoss(ignore_index = -1).to(self.device)
        self.ab_criterion = nn.BCEWithLogitsLoss().to(self.device)


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss_geno, avg_epoch_loss_pheno = self.train(self.current_epoch)
            self.train_losses_geno.append(avg_epoch_loss_geno) 
            self.train_losses_ab.append(avg_epoch_loss_pheno)  
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses_geno.append(val_results[0])
            self.val_losses_ab.append(val_results[1])
            self.val_accs.append(val_results[2])
            if self.wandb_mode:
                self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break
        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.best_epoch,
            "geno_train_losses": self.train_losses_geno,
            "ab_train_losses": self.train_losses_ab,
            "geno_val_losses": self.val_losses_geno,
            "ab_val_losses": self.val_losses_ab,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):

        self.train_losses_geno = []
        self.train_losses_ab = []

        self.val_losses_geno = []
        self.val_losses_ab = []

        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses_ab[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses_ab[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss_geno = 0
        epoch_loss_pheno = 0

        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask, AB_idx, SR_class = batch
            
            ABinclusion = torch.unique(AB_idx)
            ABinclusion = ABinclusion[ABinclusion != -1]
            ABinclusion = ABinclusion.tolist()
            #self.model.exclude_networks(ABinclusion)

            self.optimizer.zero_grad() 

            token_predictions, resistance_predictions = self.model(input, attn_mask) 
            geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
            
            result_list = []
            for j in range(len(AB_idx)):
                result_tensor = torch.full((81,), -1) 
                for idx, value in enumerate(AB_idx[j]):
                    if value != -1:
                        result_tensor[value.item()] = SR_class[j][idx]
                result_list.append(result_tensor)
            ab_loss = 0
            pheno_loss = 0
            for i, row in enumerate(resistance_predictions):
                prediction = row
                target = result_list[i]
                ab_loss = custom_loss(prediction, target.float()) 
                pheno_loss += ab_loss
            pheno_loss.backward() 
            epoch_loss_geno += geno_loss.item()
            epoch_loss_pheno += pheno_loss.item()

            self.optimizer.step()
            #self.model.reset_exclusion()   
              

        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_pheno = epoch_loss_pheno / self.num_batches

        return avg_epoch_loss_geno, avg_epoch_loss_pheno

    def custom_loss(logits, targets, pad_index=-1):
        loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        
        mask = (targets != pad_index).float()
        masked_loss = loss * mask
        
        average_loss = masked_loss.sum() / mask.sum()
        
        return average_loss

    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss_geno = 0
        epoch_loss_ab = 0
        total_correct = 0
        total_sum = 0
  
        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask, AB_idx, SR_class = batch

                token_predictions, resistance_predictions = self.model(input, attn_mask) 
                geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
                
                result_list = []
                for j in range(len(AB_idx)):
                    result_tensor = torch.full((81,), -1)  # Create tensor filled with -1 values
                    for idx, value in enumerate(AB_idx[j]):
                        if value != -1:
                            result_tensor[value.item()] = SR_class[j][idx]
                    result_list.append(result_tensor)
                ab_loss = 0
                pheno_loss = 0
                for i, row in enumerate(resistance_predictions):
                    prediction = row
                    target = result_list[i]
                    ab_loss = custom_loss(prediction, target.float()) 
                    pheno_loss += ab_loss
                epoch_loss_geno += geno_loss.item()
                epoch_loss_ab += pheno_loss.item() 
                
                list_AB_predictions = []
                pred_res = torch.where(resistance_predictions > 0, torch.ones_like(resistance_predictions), torch.zeros_like(resistance_predictions))

                for i, row in enumerate(pred_res):
                    AB_list = 0
                    AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                    current_abs = []
                    for ab in AB_list:
                        current_abs.append(row[ab].item())
                    current_abs = torch.tensor(current_abs)
                    current_abs = current_abs.type(torch.int16)
                    list_AB_predictions.append(current_abs)
                
                    processed_tensor = [row[row != -1] for row in SR_class]
                for i, row in enumerate(processed_tensor):
                    total_correct += (row == list_AB_predictions[i]).sum().item()
                    total_sum += len(row)

        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_ab = epoch_loss_ab / self.num_batches

        accuracy = total_correct / total_sum

        return avg_epoch_loss_geno, avg_epoch_loss_ab, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_embedding,
                'ff_dim': self.model.dim_embedding,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab_geno),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("GenoLosses/geno_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("GenoLosses/geno_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("AB_Losses/ab_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("AB_Losses/ab_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("Accuracies/val_acc", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            
            "GenoLosses/geno_train_loss": self.train_losses_geno[-1],
            "ABLosses/ab_train_loss": self.train_losses_ab[-1],

            "GenoLosses/geno_val_loss": self.val_losses_geno[-1],
            "ABLosses/ab_val_loss": self.val_losses_ab[-1],
            
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)

In [ ]:
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset
include_pheno = False

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)


include_pheno = False
max_length = [51,81]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [ ]:
from bert_builder import BERT_pt
from trainer import BertTrainer_pt

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERT_pt(vocab_size=len(vocabulary_geno), max_length=51, dim_embedding = 128, attention_heads=8, num_encoders=2, dropout_prob=0.2)
save_directory = 'c:\\Users\\erika\\Desktop\\Exjobb\\savefiles'

trainer = BertTrainer_pt(model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, save_directory)
trainer()

In [5]:
from bert_builder import BERT_ft
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lr = 0.001

include_pheno = True

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

#reduced_samples = 4000
#NCBI = NCBI.head(reduced_samples)

print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of token vocabulary:",len(vocabulary_geno))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)


Data loaded, 6483 samples found
length  of token vocabulary: 475


In [42]:
lr = 0.00001
bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

test = BertTrainer_ft(bert_test, train_set, val_set, 50, batch_size, lr, device, stop_patience,  True, "Trial_runs", "MaxSamples_updatedAcc_NonFrozen")
test()

ABLosses/ab_train_loss,█▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ABLosses/ab_val_loss,█▆▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Accuracies/val_acc,▁▃▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████████████
GenoLosses/geno_train_loss,▁▄ ▇█▇ ▇██▇▇ ▆▅▅▅ ▅▅▅▄▄▅ ▄▅▄▄▄▄▄▃ ▄▃▃▄▃
GenoLosses/geno_val_loss,▁▄▇███████▇█▇▇▇▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▅▅▄▄▄▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
ABLosses/ab_train_loss,9.59987
ABLosses/ab_val_loss,2.18883
epoch,50


Problem at: C:\Users\erikw\AppData\Local\Temp\ipykernel_11948\2021358228.py 223 _init_wandb


KeyboardInterrupt: 

------------------------------

In [6]:
import os
os.getcwd()

'c:\\Users\\erikw\\Desktop\\Exjobb kod\\base'

In [8]:
import os
os.getcwd()
os.chdir('c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base')
%run -i 'run.py'

C:\Users\erika\AppData\Roaming\Python\Python39\site-packages\IPython\core\magics\execution.py:726: UserWarning: For Windows, use double quotes to wrap a filename: %run "mypath\myfile.py"
  warn('For Windows, use double quotes to wrap a filename: %run "mypath\\myfile.py"')


Exception: File `"'run.py'"` not found.